# RunnableLambda 객체 사용방법 실습

In [1]:
from dotenv import load_dotenv
import os
load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_core.runnables import RunnableLambda

In [3]:
# 함수 정의
def func(x):
    return x * 2

In [4]:
# 함수 전달 인자로 func <- 함수 넣기
r1 = RunnableLambda(func)

In [5]:
input_data = 10
r1.invoke(input_data)

20

In [6]:
RunnableLambda(lambda x: x * 2)
r1.invoke(10)

20

In [7]:
# runnable batch
r3 = RunnableLambda(func)
input_data = [10, 20, 30]

r3.batch(input_data)

[20, 40, 60]

## 순차적으로 실행하게 하기

In [8]:
from langchain_core.runnables import RunnableLambda
r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 5)

chain = r1 | r2

chain.invoke(10)

35

In [9]:
from langchain_core.runnables import RunnableLambda, RunnableSequence

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 5)

# chain = r1 | r2  # 아래와 같은 표현
chain = RunnableSequence(r1, r2)

chain.invoke(10)

35

In [10]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 5)

# 병렬처리 , r1, r2 키는 임으로 설정해 주면 됨.
# chain = RunnableParallel(r1=r1, r2=r2)
chain = RunnableParallel(first=r1, second=r2)

chain.invoke(10)

{'first': 30, 'second': 15}

In [11]:
[{"foo":2}]*3

[{'foo': 2}, {'foo': 2}, {'foo': 2}]

In [12]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

runnable_1 = RunnableLambda(lambda x: {"foo": x})
runnable_2 = RunnableLambda(lambda x: [x] * 3)
runnable_3 = RunnableLambda(lambda x: str(x))

chain = runnable_1 | RunnableParallel(r2=runnable_2, r3=runnable_3)

print(chain.invoke(2))
chain.get_graph().print_ascii()

{'r2': [{'foo': 2}, {'foo': 2}, {'foo': 2}], 'r3': "{'foo': 2}"}
        +-------------+        
        | LambdaInput |        
        +-------------+        
               *               
               *               
               *               
          +--------+           
          | Lambda |           
          +--------+           
               *               
               *               
               *               
   +----------------------+    
   | Parallel<r2,r3>Input |    
   +----------------------+    
          *         *          
        **           **        
       *               *       
+--------+          +--------+ 
| Lambda |          | Lambda | 
+--------+          +--------+ 
          *         *          
           **     **           
             *   *             
  +-----------------------+    
  | Parallel<r2,r3>Output |    
  +-----------------------+    


# LCEL 문법 적용

In [13]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

# Gemini LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    temperature=0.1,
    google_api_key=os.environ["GOOGLE_API_KEY"],
)

In [14]:
# Prompt (원래 구조 그대로)
prompt = ChatPromptTemplate.from_template(
    """다음 컨텍스트에 대해서만 답변해요.
컨텍스트:
{context}

질문:
{query}
"""
)

# LCEL
chain = prompt | llm

input_data = {
    "context": "Vector Search에 의한 컨텍스트 내용",
    "query": "안녕"
}

print(chain.invoke(input_data).content)

제공된 컨텍스트는 'Vector Search에 의한 컨텍스트 내용'이라고만 설명하고 있으며, 실제 구체적인 내용이 포함되어 있지 않습니다.

따라서 이 컨텍스트만으로는 '안녕'이라는 질문에 답변할 수 없습니다.
